In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# cd ../input/pos-prsian
pwd

'/content'

In [0]:
POStr = open('drive/My Drive/Colab Notebooks/Data/POStr.txt','r').read()
train_tuples = []
dataset=[]
for data in POStr.strip().split('\n'):
  t = tuple(data.split('\t\t'))
  train_tuples.append(t)
dataset.insert(len(dataset), train_tuples)

In [0]:
# dataset => test_pos + train_pos

In [0]:
POSte = open('drive/My Drive/Colab Notebooks/Data/POSte.txt','r').read()
test_tuples = []
dataset=[]
for data in POSte.strip().split('\n'):
  t = tuple(data.split('\t\t'))
  test_tuples.append(t)
dataset.insert(len(dataset), test_tuples)

In [0]:
from collections import Counter

train_set = train_tuples
test_set = test_tuples
# tagged_words = [tup for tup in train_set]
tags = [pair[1] for pair in train_set]
unique_tags = set(tags)
tag_counts = Counter(tags)
print("We have ", len(tags), " tags.")
print("We have ", len(unique_tags), " unique tags.")
print("{ Most Common Tags: \n ", tag_counts.most_common(5), " }")


We have  195065  tags.
We have  19  unique tags.
{ Most Common Tags: 
  [('N', 81520), ('P', 24457), ('ADJ', 24410), ('DELM', 19654), ('V', 17035)]  }


In [0]:
# tokens 
tokens = [pair[0] for pair in train_set]
V = set(tokens)
T = set([pair[1] for pair in train_set])
print("\t size of vocabaulary in train set is: ", len(V))
print("\t Number of tags in train set is: ", len(T))

	 size of vocabaulary in train set is:  15322
	 Number of tags in train set is:  19


In [0]:
# computing P(w/t) and storing in T x V matrix
import numpy as np
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))
print(w_given_t.shape)

(19, 15322)


In [0]:
from collections import Counter
print(Counter([pair for pair in train_set]).most_common(10))

[(('و', 'CON'), 7760), (('.', 'DELM'), 7316), (('در', 'P'), 6826), (('به', 'P'), 5491), (('،', 'DELM'), 5458), (('از', 'P'), 4397), (('كه', 'CON'), 3957), (('است', 'V'), 3267), (('اين', 'DET'), 2470), (('را', 'P'), 2403)]


In [0]:
# لیست تگ هایی که با تگ گرفته شده برابر است را بدست آورده از بین آن ها لیست آنهایی که کلمه اش با کلمه داده شده برابر است را بذست میآوریم. خروجی تعداد ایندو است

In [0]:
# P(Word|Tag)
def word_given_tag(word, tag, train_bag = train_set):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)
print(word_given_tag('هوا', 'N'))


(46, 81520)


In [0]:
# Transition Probability

def t2_given_t1(t2, t1, train_bag = train_set):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1, count_t2_t1/count_t1)
print(t2_given_t1('N', 'P'))
print(t2_given_t1('P', 'N'))
print(t2_given_t1('P', 'P'))

(19298, 24457, 0.7890583473034305)
(9770, 81520, 0.11984789008832189)
(1053, 24457, 0.043055158032465145)


**Transaction Matrix**

In [0]:
import pandas
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[2]
tags_df = pandas.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

,MS,P,IF,SPEC,QUA,PP,MQUA,0,ADJ,PS,PRO,DELM,DET,CON,ADV,V,INT,MORP,N
MS,0.000000,0.073684,0.000000,0.000000,0.000000,0.005263,0.000000,0.000000,0.131579,0.000000,0.000000,0.284211,0.000000,0.100000,0.005263,0.105263,0.000000,0.000000,0.294737
P,0.001022,0.043055,0.000082,0.001554,0.015456,0.000041,0.000041,0.000000,0.024737,0.000000,0.045468,0.005152,0.049025,0.004866,0.005315,0.015088,0.000000,0.000041,0.789058
IF,0.005587,0.173184,0.000000,0.000000,0.013966,0.000000,0.000000,0.000000,0.047486,0.000000,0.039106,0.005587,0.041899,0.013966,0.027933,0.050279,0.000000,0.000000,0.581006
SPEC,0.006920,0.003460,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010381,0.000000,0.000000,0.003460,0.000000,0.000000,0.000000,0.003460,0.000000,0.003460,0.968858
QUA,0.000000,0.100153,0.000000,0.030581,0.000000,0.000000,0.000000,0.000000,0.009939,0.000000,0.015291,0.000765,0.017584,0.000000,0.000000,0.000000,0.000000,0.000765,0.824924
PP,0.000000,0.000000,0.000000,0.000000,0.072464,0.000000,0.000000,0.000000,0.173913,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.246377,0.000000,0.014493,0.492754
MQUA,0.000000,0.113208,0.000000,0.000000,0.094340,0.000000,0.000000,0.000000,0.000000,0.000000,0.018868,0.018868,0.000000,0.018868,0.000000,0.000000,0.000000,0.000000,0.735849
0,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.933333
ADJ,0.000041,0.196436,0.000246,0.000451,0.003892,0.000246,0.000287,0.000000,0.092462,0.000000,0.016592,0.076362,0.008316,0.103851,0.013560,0.244080,0.000000,0.000082,0.243097
PS,0.000000,0.619048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.380952,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


**Viterby**

In [0]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_set):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['0', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    print(state)
    return list(zip(words, state))

In [0]:
# list of untagged words
test_tagged_words = [tup[0] for tup in test_set]
print(test_set[:4])
print(test_tagged_words)


[('سيم', 'N'), ('پيچ', 'N'), (')', 'DELM'), ('بوبين', 'N')]
['سيم', 'پيچ', ')', 'بوبين', '(', 'مي تواند', 'ورود', 'آب', 'و', 'يا', 'قطع', 'جريان', 'آب', 'را', 'به', 'عهده', 'داشته', 'باشد', '.', 'وظيفه', 'آن', 'تؤمين', 'آب', 'دستگاه', 'مي باشد', 'كه', 'از', 'طريق', 'تايمر', 'و', 'هيدرو استات', 'فرمان', 'گرفته', 'و', 'وظيفه', 'خود', 'را', 'انجام', 'مي دهد', '.', '4', '-', 'پمپ', 'آب', ':', 'تنها', 'وسيله اي', 'است', 'كه', 'در', 'دستگاه', 'ظرفشويي', 'با', 'دامنه', 'تغييرات', 'مختلف', 'به', 'كار', 'گرفته', 'مي شود', '.', 'اين', 'پمپ', 'وظيفه اش', 'آن', 'است', 'كه', 'حركت', 'آب', 'را', 'در', 'كلكتوري', 'كه', 'توزيع', 'آب', 'را', 'روي', 'ظرف ها', 'مي نمايد', 'تعيين', 'نمايد', 'به', 'اين', 'صورت', 'كه', 'آب', 'را', 'از', 'مخزن', 'مكش', 'نموده', 'و', 'به', 'كمك', 'كلكتور', 'روي', 'ظروف', 'چرك', ')', 'چرب', '(', 'با', 'فشار', 'لازم', 'پودر', 'مي نمايد', '.', 'ساختمان', 'داخلي', 'اين', 'پمپ', 'مانند', 'ساير', 'پمپهاي', 'سانتريفوژ', 'مي باشد', 'و', 'از', 'محفظه', 'پروانه', 'و', 'موتور', 'الكتريك

In [0]:
%%time
tagged_seq = Viterbi(test_tagged_words[1:100])

['N', 'DELM', 'MS', 'DELM', 'V', 'N', 'N', 'CON', 'CON', 'N', 'N', 'N', 'P', 'P', 'N', 'ADJ', 'V', 'DELM', 'N', 'PRO', 'N', 'N', 'N', 'V', 'CON', 'P', 'N', 'N', 'CON', 'MS', 'N', 'ADJ', 'CON', 'N', 'PRO', 'P', 'N', 'V', 'DELM', 'N', 'DELM', 'N', 'N', 'DELM', 'ADV', 'ADJ', 'V', 'CON', 'P', 'N', 'N', 'P', 'N', 'N', 'ADJ', 'P', 'N', 'ADJ', 'V', 'DELM', 'DET', 'N', 'MS', 'N', 'V', 'CON', 'N', 'N', 'P', 'P', 'MS', 'CON', 'N', 'N', 'P', 'N', 'MS', 'V', 'N', 'V', 'P', 'DET', 'N', 'CON', 'N', 'P', 'P', 'N', 'N', 'ADJ', 'CON', 'P', 'N', 'MS', 'P', 'N', 'ADJ', 'DELM', 'ADJ']
CPU times: user 40.5 s, sys: 18.8 ms, total: 40.5 s
Wall time: 40.6 s


In [0]:
tagged_seq

In [0]:
# print("Time taken in seconds: ", difference)
print(tagged_seq)
print()
print(test_set[1:1000])

[('پيچ', 'N'), (')', 'DELM'), ('بوبين', 'MS'), ('(', 'DELM'), ('مي تواند', 'V'), ('ورود', 'N'), ('آب', 'N'), ('و', 'CON'), ('يا', 'CON'), ('قطع', 'N'), ('جريان', 'N'), ('آب', 'N'), ('را', 'P'), ('به', 'P'), ('عهده', 'N'), ('داشته', 'ADJ'), ('باشد', 'V'), ('.', 'DELM'), ('وظيفه', 'N'), ('آن', 'PRO'), ('تؤمين', 'N'), ('آب', 'N'), ('دستگاه', 'N'), ('مي باشد', 'V'), ('كه', 'CON'), ('از', 'P'), ('طريق', 'N'), ('تايمر', 'N'), ('و', 'CON'), ('هيدرو استات', 'MS'), ('فرمان', 'N'), ('گرفته', 'ADJ'), ('و', 'CON'), ('وظيفه', 'N'), ('خود', 'PRO'), ('را', 'P'), ('انجام', 'N'), ('مي دهد', 'V'), ('.', 'DELM'), ('4', 'N'), ('-', 'DELM'), ('پمپ', 'N'), ('آب', 'N'), (':', 'DELM'), ('تنها', 'ADV'), ('وسيله اي', 'ADJ'), ('است', 'V'), ('كه', 'CON'), ('در', 'P'), ('دستگاه', 'N'), ('ظرفشويي', 'N'), ('با', 'P'), ('دامنه', 'N'), ('تغييرات', 'N'), ('مختلف', 'ADJ'), ('به', 'P'), ('كار', 'N'), ('گرفته', 'ADJ'), ('مي شود', 'V'), ('.', 'DELM'), ('اين', 'DET'), ('پمپ', 'N'), ('وظيفه اش', 'MS'), ('آن', 'N'), ('است', '

In [0]:
# accuracy of vanilla viterbi
check = [i for i, j in zip(tagged_seq, test_set[1:1000]) if i == j] 
print(check)
accuracy = len(check)/len(tagged_seq)

accuracy

[('پيچ', 'N'), (')', 'DELM'), ('(', 'DELM'), ('مي تواند', 'V'), ('ورود', 'N'), ('آب', 'N'), ('و', 'CON'), ('يا', 'CON'), ('قطع', 'N'), ('جريان', 'N'), ('آب', 'N'), ('را', 'P'), ('به', 'P'), ('عهده', 'N'), ('داشته', 'ADJ'), ('باشد', 'V'), ('.', 'DELM'), ('وظيفه', 'N'), ('آن', 'PRO'), ('تؤمين', 'N'), ('آب', 'N'), ('دستگاه', 'N'), ('مي باشد', 'V'), ('كه', 'CON'), ('از', 'P'), ('طريق', 'N'), ('تايمر', 'N'), ('و', 'CON'), ('فرمان', 'N'), ('و', 'CON'), ('وظيفه', 'N'), ('خود', 'PRO'), ('را', 'P'), ('انجام', 'N'), ('مي دهد', 'V'), ('.', 'DELM'), ('4', 'N'), ('-', 'DELM'), ('پمپ', 'N'), ('آب', 'N'), (':', 'DELM'), ('است', 'V'), ('كه', 'CON'), ('در', 'P'), ('دستگاه', 'N'), ('ظرفشويي', 'N'), ('با', 'P'), ('دامنه', 'N'), ('تغييرات', 'N'), ('مختلف', 'ADJ'), ('به', 'P'), ('كار', 'N'), ('گرفته', 'ADJ'), ('مي شود', 'V'), ('.', 'DELM'), ('اين', 'DET'), ('پمپ', 'N'), ('آن', 'PRO'), ('است', 'V'), ('كه', 'CON'), ('حركت', 'N'), ('آب', 'N'), ('را', 'P'), ('در', 'P'), ('كه', 'CON'), ('توزيع', 'N'), ('آب', 'N

0.9309309309309309

In [0]:
predicted_tags = [pair[1] for pair in tagged_seq]
real_tags = [pair[1] for pair in test_set[1:1000]]

In [0]:
from sklearn.metrics import confusion_matrix

c_matrix = confusion_matrix(y_true=real_tags, y_pred=predicted_tags)
print("Original Confusion Matrix:\n", c_matrix)
print()
normalized_matrix = (c_matrix/(c_matrix.astype(np.float).sum(axis=1)))

print("Normalized Confusion Matrix:\n", np.around(normalized_matrix,1))

Original Confusion Matrix:
 [[ 90   1   0   0   0   0   8   0   0   0   0   0   0]
 [  1  10   0   0   0   0   1   0   0   0   0   0   0]
 [  0   0  61   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0 125   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  12   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   4   0   0   0   0   0   0   0]
 [ 45   0   0   0   0   0 400   1   0   0   0   0   0]
 [  2   0   0   0   0   0   2 134   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   1   0   0   0   0]
 [  0   0   0   0   1   0   0   0   0   9   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   4   0   0]
 [  0   0   0   0   0   0   2   0   0   0   0   1   0]
 [  5   0   0   0   0   0   0   0   0   0   0   0  79]]

Normalized Confusion Matrix:
 [[0.9 0.1 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.8 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0. 